In [4]:
from gsmmutils.model import MyModel
import seaborn as sns
sns.set(rc={'figure.figsize':(35,8.27)})
DATA_PATH = r"../data"
from os.path import join
from utils.utils import get_ps_params_multi
from cobra.flux_analysis import pfba

In [362]:
# ds = MyModel(join(DATA_PATH, 'models/model_ds.xml'), 'e_Biomass__cytop')
# ds.set_prism_reaction("PRISM_solar_litho__extr")
# ds.exchanges.EX_C00205__dra.bounds = (-20000, 20000)
# print(ds.slim_optimize())
ng = MyModel(join(DATA_PATH, 'models/model_ng.xml'), 'e_Biomass__cytop')
ng.exchanges.EX_C00205__dra.bounds = (-20000, 20000)
ng.set_prism_reaction("PRISM_solar_litho__extr")
print(ng.slim_optimize())
# pl = MyModel(join(DATA_PATH, 'models/model_pl.xml'), 'e_Biomass__cytop')
# pl.exchanges.EX_C00205__dra.bounds = (-20000, 20000)
# pl.set_prism_reaction("PRISM_fluorescent_cool_215W__extr")
# pl.reactions.e_Biomass_ht__cytop.bounds = (0,0)
# print(pl.slim_optimize())

Loading

Reactions: 4433
Metabolites: 3769
Genes: 1333
Model loaded
0.407156324014842


In [3]:
get_ps_params_multi({"Dsalina": ds, "Ngadiitana": ng, "Plutheri": pl})

,QY,PQ
Dsalina,0.037,1.243
Ngadiitana,0.032,1.409
Plutheri,0.028,1.416


In [389]:
from light_absorption import update_model

with ng as model:
    # model.reactions.R09502__chlo.bounds = (0, 0)
    model.set_prism_reaction("PRISM_solar_litho__extr")
    coeff = sum(e for e in model.reactions.PRISM_solar_litho__extr.metabolites.values() if e > 0)
    for rxn in model.reactions:
        if rxn.lower_bound <= -1000:
            rxn.lower_bound = -20000
        if rxn.upper_bound >= 1000:
            rxn.upper_bound = 20000
    same_flux = model.problem.Constraint(
        model.reactions.R00024__chlo.flux_expression * 0.025 - model.reactions.R03140__chlo.flux_expression,
        lb=-1000,
        ub=0)

    max_cef_lef_ratio = 0.1  # sol.fluxes['CEF__chlo'] / (sol.fluxes['R01195__chlo'] + sol.fluxes['CEF__chlo']) 
    cef_constrain = model.problem.Constraint(
        model.reactions.CEF__chlo.flux_expression * (1 - max_cef_lef_ratio) - model.reactions.R01195__chlo.flux_expression * max_cef_lef_ratio,
        lb=-1000,
        ub=0)
    ue_m2s = 1000
    max_npq = (ue_m2s * 0.0574+33.556) /100
    if max_npq > 0.75: max_npq = 0.75
    npq_constrain = model.problem.Constraint(
        model.reactions.CHLAPSIIdex__chlo.flux_expression * (1-max_npq) - model.reactions.PSIICSa__chlo.flux_expression * max_npq,
        lb=-1000,
        ub=0)
    model.add_cons_vars([cef_constrain, npq_constrain])
    max_photosynthesis = {"dsalina": 177.8, "ngaditana": 136.5, "plutheri": 44.7}
    step_size = {"dsalina": 50, "ngaditana": 50, "plutheri": 30}
    model.reactions.PSII__lum.bounds = (0, max_photosynthesis["ngaditana"])
    npq = {}
    # uptake = range(int(round(abs(min_val), 0)), int(round(max_val, 0)), 100)
    ros_reactions = {"FLV__chlo", "CEF__chlo",
                     # "NGAM_D1__chlo",
                     "R12570__chlo", "R09540__chlo",
                     "R00274__chlo", "R00017__mito",
                     "PSII__lum", "R01195__chlo", "PSI__lum", "PSIc6__lum"
                     }
    photosystems = {"PSII__lum", "R01195__chlo", "PSI__lum", "PSIc6__lum"}
    ros_reactions = {key: [0, 0, 0] for key in ros_reactions if key in model.reaction_ids}
    ros_results_by_light = {}
    ros_results_by_reaction = {}
    conversion_factor = 8.33
    mmol_gdwd = conversion_factor * ue_m2s
    model = update_model(model, 'ngaditana', ue_m2s)
    model.exchanges.EX_C00205__dra.bounds = (-mmol_gdwd, -mmol_gdwd)
    ngam = ue_m2s * 0.022 + 6.85
    # model.reactions.NGAM__lum.bounds = (ngam, ngam)
    model.reactions.NGAM__lum.bounds = (0, 1000)
    model.reactions.ATPm__cytop.bounds = (0, 1000)
    model.exchanges.EX_C00011__dra.bounds = (-1000, 1000)
    model.create_sink("C00138__chlo")
    model.create_sink("C00139__chlo")
    sol = model.maximize(value=False)
    print(model.metabolites.C16695__lum.summary(sol))
    # print(model.metabolites.C10385__lum.summary(sol, fva=0.99))
    # print(model.reactions.CEF__chlo.summary(fva=1.0))

C16695__lum
Formula: C53H82O2

Producing Reactions
-------------------
Percent  Flux  Reaction                                                                      Definition
 67.33% 562.5 CEF__chlo 2.0 C00080__chlo + 2.0 C00138__chlo + C10385__lum --> 2.0 C00139__chlo + C16...
 32.67%   273 PSII__lum 2.0 C00001__lum + 3.9996 C00080__chlo + 1.9998 C10385__lum + 4.0 p680_exc__l...

Consuming Reactions
-------------------
Percent   Flux         Reaction                                                                      Definition
100.00% -835.5 RXN1YI0_20__chlo 2.0 C00080__chlo + C16695__lum + 2.0 MNXM1136552__lum --> 4.0 C00080__lum + ...


In [370]:
sol.fluxes['CEF__chlo'] / (sol.fluxes['R01195__chlo'] + sol.fluxes['CEF__chlo'])

0.0

In [371]:
sol.fluxes['CHLAPSIIdex__chlo'] / (sol.fluxes['PSIICSa__chlo']+sol.fluxes['CHLAPSIIdex__chlo'])

C:\Users\Bisbii\AppData\Local\Temp\ipykernel_41384\2472400463.py:1: RuntimeWarning: invalid value encountered in scalar divide
  sol.fluxes['CHLAPSIIdex__chlo'] / (sol.fluxes['PSIICSa__chlo']+sol.fluxes['CHLAPSIIdex__chlo'])


nan

In [358]:
sol.fluxes['CHLAPSIdex__chlo'] / (sol.fluxes['PSICSa__chlo']+sol.fluxes['CHLAPSIdex__chlo'])

0.5160305753674432

In [266]:
5148/(5148+1716)

0.75

In [359]:
sol.fluxes['DM_pho_loss__chlo'] / (abs(sol.fluxes['EX_C00205__dra']) * coeff)

0.8606476133807025

In [146]:
ng.metabolites.C16695__lum.summary(solution=sol)

Percent,Flux,Reaction,Definition
100.00%,1011,PSII__lum,2.0 C00001__lum + 3.9996 C00080__chlo + 1.9998 C10385__lum + 4.0 p680_exc__lum + 0.0004 ps2d1__lum --> C00007__lum + 4.0 C00080__lum + 1.9998 C16695__lum + 4.0 p680__lum + 0.0004 ps2d1_exc__lum
Percent,Flux,Reaction,Definition
5.71%,-57.7,NGAM__lum,C00007__lum + 2.0 C16695__lum --> 2.0 C00001__lum + 2.0 C10385__lum
94.29%,-953.1,RXN1YI0_20__chlo,2.0 C00080__chlo + C16695__lum + 2.0 MNXM1136552__lum --> 4.0 C00080__lum + C10385__lum + 2.0 MNXM1136754__lum
